# vectorDBpipe: Omni-RAG Architecture Demo

This notebook demonstrates the core capabilities of the `VDBpipe` package running locally. We will ingest some complex dummy data using the new **Tri-Processing Engine** and test out the **4 Engine Routing System**.

### 1. Setup & Dummy Data Generation
First, let's create a dummy contract and a dummy financial report to represent unstructured enterprise knowledge.

In [2]:
pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ---------------------------------------- 0.0/161.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/161.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/161.7 MB ? eta -:--:--
     ---------------------------------------- 0.3/161.7 MB ? eta -:--:--
     ---------------------------------------- 0.5/161.7 MB 1.4 MB/s eta 0:01:55
     ---------------------------------------- 1.3/161.7 MB 2.2 MB/s eta 0:01:15
      --------------------------------------- 2.1/161.7 MB 2.7 MB/s eta 0:00:59
      --------------------------------------- 3.4/161.7 MB 3.6 MB/s eta 0:00:45
     - -------------------------------------- 4.7/161.7 MB 4.0 MB/s eta 0:00:40
     - -------------------------------------- 5.5/161.7 MB 4.0 MB/s eta 0:00:40
     - -------------------------------------- 7.3/161.7 MB 4.6 MB/s eta 0:00:34
     -- ------------------------------------- 8.7/161.7 MB 4.8 MB/s eta 0:00:32
     -- ---

  You can safely remove it manually.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
# Check if the module is imported and print its version
print(f"PyTorch is successfully imported!")
print(f"Current PyTorch Version: {torch.__version__}")

PyTorch is successfully imported!
Current PyTorch Version: 2.3.1+cpu


In [ ]:
pip install -r "requirements.txt"

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [1]:
print("Hello")

Hello


In [2]:
import os

# Ensure our data directory exists
os.makedirs("demo_data", exist_ok=True)

dummy_contract = """
# MASTER SERVICE AGREEMENT: ACME CORP & GLOBEX INC

## 1. Introduction
This agreement bounds Acme Corp (the 'Client') and Globex Inc (the 'Vendor') for the provision of cloud infrastructure services.
The CEO of Acme Corp, Jane Doe, has personally signed this agreement alongside John Smith, VP of Sales at Globex.

## 2. Payment Terms
Invoices are generated on the 1st of every month. The standard payment is $50,000 USD per month.
Late Payment Penalty: If the Client fails to pay within 30 days of the invoice date, a strict penalty of 5% of the outstanding balance will be applied daily. This penalty is non-negotiable and legally binding.

## 3. Termination
Either party may terminate this agreement with a 90-day written notice. Immediate termination is permitted if Acme Corp enters bankruptcy or if Globex Inc fails to maintain 99.9% server uptime.
"""

with open("demo_data/contract.md", "w", encoding="utf-8") as f:
    f.write(dummy_contract)
    
print("✅ Dummy data created at `demo_data/contract.md`")

✅ Dummy data created at `demo_data/contract.md`


### 2. Initializing the Omni-RAG Engine
We import `VDBpipe`. This acts as our router. Note: For this demo to work fully, you should have `OPENAI_API_KEY` set in your environment (or `config.yaml` configured), but the vector search will work regardless.

In [ ]:
pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cpu

In [3]:
import warnings
warnings.filterwarnings('ignore')

from vectorDBpipe import VDBpipe

# Initialize the pipeline (using default local config)
pipeline = VDBpipe()

# 1. INGESTION (Tri-Processing)
# Under the hood, this chunks text (Vector DB), writes a JSON Table of Contents (PageIndex), 
# and extracts relationships into a NetworkX graph simultaneously via multi-threading.
pipeline.ingest("demo_data")

Disabling PyTorch because PyTorch >= 2.4 is required but found 2.3.1+cpu
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'langchain_core.pydantic_v1'

### 3. Testing Engine 1: Traditional Vector RAG
**Use Case:** Factual Lookups.
*The router detects a specific factual query and uses cosine similarity against the chunked data.*

In [ ]:
factual_query = "What is the exact penalty for a late payment?"

print("Querying Engine 1...")
res_1 = pipeline.query(factual_query)
print("\n--- RESULT ---\n", res_1)

### 4. Testing Engine 2: Vectorless / PageIndex RAG
**Use Case:** Holistic Summarization.
*The router detects a summarization request and bypasses the vector database chunks to read the document's hierarchal JSON structure instead.*

In [ ]:
summary_query = "Summarize the termination clauses and the overall tone of the contract."

print("Querying Engine 2...")
res_2 = pipeline.query(summary_query)
print("\n--- RESULT ---\n", res_2)

### 5. Testing Engine 3: GraphRAG
**Use Case:** Complex Multi-Hop Reasoning.
*The router detects relationship questions and traverses the NetworkX Knowledge Graph generated during ingestion.*

In [ ]:
graph_query = "How is Jane Doe connected to John Smith? What companies are they associated with?"

print("Querying Engine 3...")
res_3 = pipeline.query(graph_query)
print("\n--- RESULT ---\n", res_3)

### 6. Testing Engine 4: LangChain Extract
**Use Case:** Structured Output (JSON generation).
*We explicitly call `.extract()` with a strict schema to force the LLM to return programmatic data.*

In [ ]:
schema = {
    "companies_involved": ["string"],
    "monthly_payment": "integer",
    "late_penalty_percentage": "integer"
}

extract_query = "Find the companies and the financial metrics in the agreement."

print("Querying Engine 4 (Extract)...")
data_json = pipeline.extract(extract_query, schema)

print("\n--- EXTRACTED JSON STRUCTURE ---")
import json
print(json.dumps(data_json, indent=2))

---
*Demo Complete. `vectorDBpipe` seamlessly handled the embedding orchestration, structural parsing, graph extraction, and LangChain validation via the unified `VDBpipe` router.*